In [1]:
import os, sys, glob, re
import coffea.util
sys.path.append("../../")
from plotting.plot_utils import *
from plotting import fill_utils

In [2]:
dataDir = '/data/submit/cms/store/user/lavezzo/SUEP/cutflows5/'
subDirs = os.listdir(dataDir)

In [3]:
counts = {}
for subDir in subDirs:
    files = os.listdir(dataDir + subDir)
    
    for i, f in enumerate(files):
        if i == 0: 
            cum = coffea.util.load(os.path.join(dataDir, subDir, f))
        else:
            temp = coffea.util.load(os.path.join(dataDir, subDir, f))
            for key in temp:
                cum[key] += temp[key]
                
    counts[subDir.split('+')[0]] = cum

In [4]:
counts.keys()

dict_keys(['QCD_HT500to700_TuneCP5_PSWeights_13TeV-madgraph-pythia8', 'SUEP-m1000-generic', 'SUEP-m400-generic', 'SUEP-m125-generic-htcut', 'QCD_HT100to200_TuneCP5_PSWeights_13TeV-madgraph-pythia8', 'SUEP-m750-generic', 'QCD_HT50to100_TuneCP5_PSWeights_13TeV-madgraph-pythia8', 'QCD_HT1500to2000_TuneCP5_PSWeights_13TeV-madgraph-pythia8', 'SUEP-m125-generic', 'QCD_HT300to500_TuneCP5_PSWeights_13TeV-madgraph-pythia8', 'QCD_HT2000toInf_TuneCP5_PSWeights_13TeV-madgraph-pythia8', 'QCD_HT1000to1500_TuneCP5_PSWeights_13TeV-madgraph-pythia8', 'QCD_HT200to300_TuneCP5_PSWeights_13TeV-madgraph-pythia8'])

In [5]:
counts_generic = {}
for k, i in counts.items(): 
    if 'generic' not in k: continue
    #if 'htcut' in k: continue
    
    counts_generic[k] = i
    
counts_qcd = None
for k, i in counts.items(): 
    if 'QCD_HT' not in k: continue
    
    xsection = fill_utils.getXSection(k+'+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM', 
                                      '2018', path='../../data/')

    if counts_qcd is None: 
        counts_qcd = {}
        for key in counts[k].keys():
            val = counts[k][key][list(counts[k][key].keys())[0]]
            sumw = counts[k]['sumw'][list(counts[k][key].keys())[0]]
            scale = lumis['2018'] * xsection / sumw
            counts_qcd[key] = val*scale

    else:
        for key in counts[k].keys():
            if len(list(counts[k][key])) > 0:
                val = counts[k][key][list(counts[k][key].keys())[0]]
                sumw = counts[k]['sumw'][list(counts[k][key].keys())[0]]
                scale = lumis['2018'] * xsection / sumw
                counts_qcd[key] += val * scale

In [6]:
counts_generic.keys()

dict_keys(['SUEP-m1000-generic', 'SUEP-m400-generic', 'SUEP-m125-generic-htcut', 'SUEP-m750-generic', 'SUEP-m125-generic'])

In [7]:
columnLabels = ['Selection', 'QCD', r'$M_{S}$ = 125 GeV', r'$M_{S}$ = 400 GeV', 
           r'$M_{S}$ = 750 GeV', r'$M_{S}$ = 1000 GeV']
selectionLabels = ['Total', 'Trigger (PFHT1050)', 
              r'$H_T$ > 1200 GeV', 'Two ak15 Jets',
             '# Constituents > 80', r'$Sph_1$ > 0.5']

selectionKeys = list(counts_generic[list(counts_generic.keys())[0]].keys())[1:]
columnKeys = ['QCD', 'SUEP-m125-generic-htcut', 'SUEP-m400-generic', 
              'SUEP-m750-generic', 'SUEP-m1000-generic'] 

In [8]:
import hist.intervals as intervals
import numpy as np

In [17]:
table = []
table.append(columnLabels)

raw_vals = {}
for iSel, (label, selection) in enumerate(zip(selectionLabels, selectionKeys)):
    columns = []
    columns.append(label)
    for k in columnKeys: 
        
        if iSel == 0: raw_vals.update({k:[]})
        
        if k == 'QCD':
            
            val = 0
            differences = [[],[]]
            for sample, i in counts.items(): 
                if 'QCD_HT' not in sample: continue
                if len(list(counts[sample][selection])) == 0: continue
                    
                xsection = fill_utils.getXSection(sample+'+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM', 
                                       '2018', path='../../data/')
                
                bin_val = counts[sample][selection][list(counts[sample][selection].keys())[0]]
                
                sumw = counts[sample]['sumw'][list(counts[sample][selection].keys())[0]]
                scale = lumis['2018'] * xsection / sumw

                bin_val *= scale

                val += bin_val
                
            interval = [x[0] for x in intervals.poisson_interval(np.array([val]), np.array([val]))]
            differences = [round(x - val) for x in interval]
                
        else:
            
            mass = k.split("SUEP-m")[1]
            mass = mass.split('-')[0]
            mass = str(mass)
            xsec = fill_utils.getXSection(mass, 2018, SUEP=True,path='../../data/')
            
            if 'htcut' in k: xsec *= 0.003
            
            counts_sample = counts_generic[k][selection]
            sumw_sample = counts_generic[k]['sumw']
            val = round(counts_sample[list(counts_sample.keys())[0]])
            sumw = round(sumw_sample[list(sumw_sample.keys())[0]])
            
            scale = (lumis['2018'] * xsec / sumw)
            
            interval = [x[0]*scale for x in intervals.poisson_interval(np.array([val]))]
            val *= scale
                        
            differences = [round(x - val) for x in interval]
                        
        raw_vals[k].append(val)
        columns.append('$'+str(round(val))+'^{+'+str(differences[1])+'}_{'+str(differences[0])+'}$')
    table.append(columns)

In [18]:
table

[['Selection',
  'QCD',
  '$M_{S}$ = 125 GeV',
  '$M_{S}$ = 400 GeV',
  '$M_{S}$ = 750 GeV',
  '$M_{S}$ = 1000 GeV'],
 ['Total',
  '$12729207928447^{+3567803}_{-3567802}$',
  '$6245^{+52}_{-52}$',
  '$352958^{+3583}_{-3547}$',
  '$29915^{+302}_{-299}$',
  '$10163^{+103}_{-102}$'],
 ['Trigger (PFHT1050)',
  '$46250911^{+6802}_{-6801}$',
  '$4631^{+45}_{-45}$',
  '$3957^{+412}_{-375}$',
  '$754^{+51}_{-47}$',
  '$360^{+20}_{-19}$'],
 ['$H_T$ > 1200 GeV',
  '$26186149^{+5118}_{-5117}$',
  '$3070^{+37}_{-36}$',
  '$3209^{+375}_{-338}$',
  '$616^{+46}_{-43}$',
  '$306^{+19}_{-18}$'],
 ['Two ak15 Jets',
  '$23769056^{+4876}_{-4875}$',
  '$2928^{+36}_{-36}$',
  '$3137^{+371}_{-334}$',
  '$595^{+45}_{-42}$',
  '$280^{+18}_{-17}$'],
 ['# Constituents > 80',
  '$97977^{+314}_{-313}$',
  '$176^{+9}_{-9}$',
  '$3030^{+366}_{-328}$',
  '$592^{+45}_{-42}$',
  '$279^{+18}_{-17}$'],
 ['$Sph_1$ > 0.5',
  '$47745^{+220}_{-219}$',
  '$91^{+7}_{-6}$',
  '$2817^{+354}_{-316}$',
  '$565^{+44}_{-41}$',
  '$2

In [21]:
import tabulate
t = tabulate.tabulate(table,tablefmt ='latex')

def convert2LatexString(s):
    r = r'(\^\{\})'; s = re.sub(r, "^", s)
    s = re.sub(r'\\([\$\_\{\}\^])', r'\1', s)
    s = re.sub(r'(\\textbackslash{})', r'\\', s)
    return s

t = convert2LatexString(t)
with open('table.txt', 'w') as f:
    f.write(t)
t

'\\begin{tabular}{llllll}\n\\hline\n Selection           & QCD                                    & $M_{S}$ = 125 GeV  & $M_{S}$ = 400 GeV        & $M_{S}$ = 750 GeV     & $M_{S}$ = 1000 GeV    \\\\\n Total               & $12980865195580^{+3602898}_{-3602897}$ & $6368^{+53}_{-53}$ & $359936^{+3654}_{-3617}$ & $30506^{+308}_{-305}$ & $10364^{+105}_{-104}$ \\\\\n Trigger (PFHT1050)  & $47165295^{+6869}_{-6868}$             & $4723^{+46}_{-46}$ & $4036^{+421}_{-382}$     & $769^{+52}_{-48}$     & $367^{+21}_{-19}$     \\\\\n $H_T$ \\ensuremath{>} 1200 GeV    & $26703851^{+5169}_{-5168}$             & $3130^{+38}_{-37}$ & $3272^{+383}_{-344}$     & $628^{+47}_{-44}$     & $312^{+19}_{-18}$     \\\\\n Two ak15 Jets       & $24238972^{+4924}_{-4923}$             & $2986^{+37}_{-36}$ & $3199^{+379}_{-340}$     & $607^{+46}_{-43}$     & $286^{+18}_{-17}$     \\\\\n \\# Constituents \\ensuremath{>} 80 & $99914^{+317}_{-316}$                  & $179^{+9}_{-9}$    & $3090^{+373}_{-335}$     & $6

In [56]:
rel_table = []
rel_table.append(columnLabels)

selectionKeys = list(counts_generic[list(counts_generic.keys())[0]].keys())

for iSel, (label, selection) in enumerate(zip(selectionLabels, selectionKeys)):
    
    if iSel == 0: continue
    
    columns = []
    columns.append(label)
    for k in columnKeys: 
                
        val = raw_vals[k][iSel]
        prev_val = raw_vals[k][iSel-1]
        differences = [[],[]]
        ratio = val / prev_val if prev_val > 0 else 0
        ratio_err = hist.intervals.ratio_uncertainty(np.array([val]), np.array([prev_val]))
        
        ratio *= 100
                                
        columns.append('$'+str(round(ratio,2))+'^{+'+str(round(ratio_err[1][0]*100,2))+'}_{'+str(round(ratio_err[0][0]*100,2))+'}$ %')
    rel_table.append(columns)

In [57]:
rel_table

[['Selection',
  'QCD',
  '$M_{S}$ = 125 GeV',
  '$M_{S}$ = 400 GeV',
  '$M_{S}$ = 750 GeV',
  '$M_{S}$ = 1000 GeV'],
 ['Trigger (PFHT1050)',
  '$0.0^{+0.0}_{0.0}$ %',
  '$74.16^{+1.11}_{1.09}$ %',
  '$1.12^{+0.02}_{0.02}$ %',
  '$2.52^{+0.1}_{0.09}$ %',
  '$3.54^{+0.2}_{0.19}$ %'],
 ['$H_T$ > 1200 GeV',
  '$56.62^{+0.01}_{0.01}$ %',
  '$66.28^{+1.22}_{1.2}$ %',
  '$81.08^{+1.46}_{1.43}$ %',
  '$81.75^{+3.43}_{3.29}$ %',
  '$85.03^{+5.14}_{4.86}$ %'],
 ['Two ak15 Jets',
  '$90.77^{+0.02}_{0.02}$ %',
  '$95.39^{+1.8}_{1.76}$ %',
  '$97.78^{+1.78}_{1.75}$ %',
  '$96.6^{+4.12}_{3.96}$ %',
  '$91.69^{+5.81}_{5.47}$ %'],
 ['# Constituents > 80',
  '$0.41^{+0.0}_{0.0}$ %',
  '$6.0^{+0.49}_{0.45}$ %',
  '$96.59^{+1.79}_{1.75}$ %',
  '$99.5^{+4.26}_{4.09}$ %',
  '$99.64^{+6.32}_{5.96}$ %'],
 ['$Sph_1$ > 0.5',
  '$48.73^{+0.22}_{0.22}$ %',
  '$51.72^{+6.02}_{5.42}$ %',
  '$92.94^{+1.78}_{1.75}$ %',
  '$95.45^{+4.19}_{4.01}$ %',
  '$95.27^{+6.2}_{5.83}$ %']]

In [58]:
import tabulate
t = tabulate.tabulate(rel_table,tablefmt ='latex')

def convert2LatexString(s):
    r = r'(\^\{\})'; s = re.sub(r, "^", s)
    s = re.sub(r'\\([\$\_\{\}\^])', r'\1', s)
    s = re.sub(r'(\\textbackslash{})', r'\\', s)
    return s

t = convert2LatexString(t)
with open('rel_table.txt', 'w') as f:
    f.write(t)
t

'\\begin{tabular}{llllll}\n\\hline\n Selection           & QCD                      & $M_{S}$ = 125 GeV        & $M_{S}$ = 400 GeV        & $M_{S}$ = 750 GeV        & $M_{S}$ = 1000 GeV       \\\\\n Trigger (PFHT1050)  & $0.0^{+0.0}_{0.0}$ \\%     & $74.16^{+1.11}_{1.09}$ \\% & $1.12^{+0.02}_{0.02}$ \\%  & $2.52^{+0.1}_{0.09}$ \\%   & $3.54^{+0.2}_{0.19}$ \\%   \\\\\n $H_T$ \\ensuremath{>} 1200 GeV    & $56.62^{+0.01}_{0.01}$ \\% & $66.28^{+1.22}_{1.2}$ \\%  & $81.08^{+1.46}_{1.43}$ \\% & $81.75^{+3.43}_{3.29}$ \\% & $85.03^{+5.14}_{4.86}$ \\% \\\\\n Two ak15 Jets       & $90.77^{+0.02}_{0.02}$ \\% & $95.39^{+1.8}_{1.76}$ \\%  & $97.78^{+1.78}_{1.75}$ \\% & $96.6^{+4.12}_{3.96}$ \\%  & $91.69^{+5.81}_{5.47}$ \\% \\\\\n \\# Constituents \\ensuremath{>} 80 & $0.41^{+0.0}_{0.0}$ \\%    & $6.0^{+0.49}_{0.45}$ \\%   & $96.59^{+1.79}_{1.75}$ \\% & $99.5^{+4.26}_{4.09}$ \\%  & $99.64^{+6.32}_{5.96}$ \\% \\\\\n $Sph_1$ \\ensuremath{>} 0.5       & $48.73^{+0.22}_{0.22}$ \\% & $51.72^{+6.02}_{5.